In [1]:
import os, sys, math
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [4]:

# Read Data File

data = pd.read_csv("C:/VT/Independent Study/RFnEnsamle.csv", sep='\t')
data.head() # Shows the data structure for first few rows. Peptide Column is categorical value. Target column Label_char also is a catgeorical value.



,peptide,label_chr,label_num,data_type
0,LLTDAQRIV,WB,1,train
1,LMAFYLYEV,SB,2,train
2,VMSPITLPT,WB,1,test
3,SLHLTNCFV,WB,1,train
4,RQFTCMIAV,WB,1,train


In [3]:
data.describe() # Shows summary of the numerical attributes
data.info() # There are 23760 instances in the data set.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23760 entries, 0 to 23759
Data columns (total 4 columns):
peptide      23760 non-null object
label_chr    23760 non-null object
label_num    23760 non-null int64
data_type    23760 non-null object
dtypes: int64(1), object(3)
memory usage: 742.6+ KB


In [5]:
# Prepare the Data for Machine Learning Algorithm

x = data.drop(columns=["label_num", "data_type", "label_chr"])

y = data.drop(columns=["peptide", "label_chr", "data_type"])

In [6]:
x.head()
y.head()

,label_num
0,1
1,2
2,1
3,1
4,1


In [7]:
#Peptide encoding equence using the 20 Amino acids labels tha t form a peptide bond 

codes = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L',
         'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']

In [8]:

def one_hot_encode(seq):
    o = list(set(codes) - set(seq))
    s = pd.DataFrame(list(seq))
    x = pd.DataFrame(np.zeros((len(seq),len(o)),dtype=int),columns=o)
    a = s[0].str.get_dummies(sep=',')
    a = a.join(x)
    a = a.sort_index(axis=1)
    #show_matrix(a)
    e = a.values.flatten()
    return e

In [9]:
pep = 'ALDFEQEMT'
#test one hot encoding
e = one_hot_encode(pep)
print(e)

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]


In [10]:
#apply one hot encoding on peptides

x = x.values
print("x length after removing header: " + str(len(x)))
x_loop = []
for i in x:
    x_loop.append(one_hot_encode(i))
    #print(x_loop)

print("x_loop length: " + str(len(x_loop)))

x length after removing header: 23760
x_loop length: 23760


In [11]:
x = np.asmatrix(x_loop)
y = y.as_matrix()

print(len(x))
print(len(y))

23760
23760


C:\Users\vishn\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [12]:
# Perform Data Partition Split to creat Test and Train

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=64)

In [13]:
#Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1) 

rnd_clf.fit(x_train, y_train)

test_y_pred_rf = rnd_clf.predict(x_test)

print(accuracy_score(y_test, test_y_pred_rf))

C:\Users\vishn\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


0.4346240179573513


In [14]:
#Ensamble 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import SGDClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier


dtree_clf = DecisionTreeClassifier(max_depth=6)
log_clf = LogisticRegression(multi_class='multinomial', solver = 'lbfgs', C=10, max_iter=1000)
sgd_clf = SGDClassifier(max_iter=10000, tol=1e-3)

voting_clf = VotingClassifier(
            estimators=[('dt', dtree_clf), ('lr', log_clf), ('sgd', sgd_clf)],
            voting='hard')

voting_clf.fit(x_train, y_train)

test_y_pred = voting_clf.predict(x_test)

C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:

from sklearn.metrics import accuracy_score

print(accuracy_score(y_test, test_y_pred))

0.42845117845117847


In [16]:
#Individual accuracy

for clf in (dtree_clf, log_clf, sgd_clf, voting_clf):
    clf.fit(x_train, y_train)
    test_y_pred = clf.predict(x_test)
    print(clf.__class__.__name__, accuracy_score(y_test, test_y_pred))

DecisionTreeClassifier 0.4068462401795735
LogisticRegression

C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


 0.4346240179573513
SGDClassifier 0.4024971941638608


C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


VotingClassifier 0.4260662177328844


In [17]:
#Bagging classifier

from sklearn.ensemble import BaggingClassifier 

bag_clf = BaggingClassifier( 
            SGDClassifier(), n_estimators= 70, 
            max_samples= 100, bootstrap=True, n_jobs=-1, oob_score=True) 

bag_clf.fit(x_train, y_train)
bag_clf.oob_score_

C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\ensemble\bagging.py:621: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.43566618566618565

In [18]:
#Stochastic Gradient Boosting

# Use GradientBoosting

from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(max_depth= 5, n_estimators= 500, learning_rate=0.1, subsample=0.75) 

gbclf.fit(x_train, y_train)

C:\Users\vishn\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=5,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=500,
              n_iter_no_change=None, presort='auto', random_state=None,
              subsample=0.75, tol=0.0001, validation_fraction=0.1,
              verbose=0, warm_start=False)

In [19]:
test_y_pred = gbclf.predict(x_test)

print(accuracy_score(y_test, test_y_pred))

0.4346240179573513
